In [150]:
import pandas as pd

# Load data

In [151]:
awards_players_df = pd.read_csv('../data/awards_players.csv'); awards_players_df.Name = "awards_players"
coaches_df = pd.read_csv('../data/coaches.csv'); coaches_df.Name = "coaches"
players_teams_df = pd.read_csv('../data/players_teams.csv'); players_teams_df.Name = "players_teams"
players_df = pd.read_csv('../data/players.csv'); players_df.Name = "players"
series_post_df = pd.read_csv('../data/series_post.csv'); series_post_df.Name = "series_post"
teams_post_df = pd.read_csv('../data/teams_post.csv'); teams_post_df.Name = "teams_post"
teams_df = pd.read_csv('../data/teams.csv'); teams_df.Name = "teams"

dfs = [awards_players_df, coaches_df, players_teams_df, players_df, series_post_df, teams_post_df, teams_df]

# Clean data

### Drop noisy data

In [152]:
players_df.drop(players_df[players_df['height'] <= 0].index, inplace = True)

### Remove duplicates


In [153]:
for df in dfs:
    df.drop_duplicates(inplace=True)
    display(f"Dropping dupplicates from dataframe {df.Name}...")

'Dropping dupplicates from dataframe awards_players...'

'Dropping dupplicates from dataframe coaches...'

'Dropping dupplicates from dataframe players_teams...'

'Dropping dupplicates from dataframe players...'

'Dropping dupplicates from dataframe series_post...'

'Dropping dupplicates from dataframe teams_post...'

'Dropping dupplicates from dataframe teams...'

### Drop unnecessary columns

In [154]:
players_df.drop(['birthDate', 'deathDate', 'college', 'collegeOther'], axis=1, inplace=True)
teams_df.drop(["confID", "franchID", "firstRound", "semis", "finals", "attend", "name", "arena"], axis=1, inplace=True);
teams_post_df.drop([], axis=1, inplace=True)
series_post_df.drop([], axis=1, inplace=True)
players_teams_df.drop([], axis=1, inplace=True)
awards_players_df.drop(['award'], axis=1, inplace=True)
coaches_df.drop([], axis=1, inplace=True)

### Drop single value columns

In [155]:
def drop_single_value_columns(df):
    for column in df.columns:
        if len(df[column].unique()) == 1:
            display(f"Dropping column {column} from datarame {df.Name}")

            df.drop(column, axis=1, inplace=True)

for df in dfs:
    drop_single_value_columns(df)

'Dropping column lgID from datarame awards_players'

'Dropping column lgID from datarame coaches'

'Dropping column lgID from datarame players_teams'

'Dropping column firstseason from datarame players'

'Dropping column lastseason from datarame players'

...

'Dropping column tmDRB from datarame teams'

'Dropping column tmTRB from datarame teams'

'Dropping column opptmORB from datarame teams'

'Dropping column opptmDRB from datarame teams'

'Dropping column opptmTRB from datarame teams'

### Drop rows with missing values

In [156]:
players_df.drop(players_df[players_df['height'] == 0].index, inplace = True)
players_df.drop(players_df[players_df['weight'] == 0].index, inplace = True)

### Rename columns and replace values

In [157]:
players_df.rename(columns={'bioID': 'playerID', 'stint': 'player_stint'}, inplace=True)
players_df.replace('F-C', 'C-F', inplace=True)
players_df.replace('F-G', 'G-F', inplace=True)

coaches_df.rename(columns={'won': 'coach_wins', 'lost': 'coach_losses', 'post_wins': 'coach_post_wins', 'post_losses': 'coach_post_losses', 'stint': 'coach_stint'}, inplace=True)

teams_df.rename(columns={'GP': 'team_GP', 'rank': 'current_year_rank'}, inplace=True)

players_teams_df.rename(columns={'GP': 'player_team_GP'}, inplace=True)

teams_post_df.rename(columns={'W': 'team_post_wins', 'L': 'team_post_losses'}, inplace=True)

series_post_df.rename(columns={'W': 'series_post_wins', 'L': 'series_post_lost'}, inplace=True)

### Add new columns

In [158]:
awards_players_df['award'] = True

for tmID, team_data in teams_df.groupby('tmID'):
    for year in team_data['year'].values:
        # if year < 2: continue

        previous_year = year - 1

        previous_year_data = team_data[team_data['year'] == previous_year]

        # register previous year rank for each team
        previous_year_rank_series_data = previous_year_data['current_year_rank'].values
        previous_year_rank = previous_year_rank_series_data[0] if len(previous_year_rank_series_data) > 0 else -1
        teams_df.loc[((teams_df['tmID'] == tmID) & (team_data['year'] == year)), 'previous_year_rank'] = previous_year_rank

        # register average height and weight for each position for each team
        previous_year_player_ids = players_teams_df[(players_teams_df['tmID'] == tmID) & (players_teams_df['year'] == previous_year)]['playerID'].values
        previous_year_players = players_df[players_df['playerID'].isin(previous_year_player_ids)]
        for position, players_per_position in previous_year_players.groupby('pos'):

            previous_year_avg_height = players_per_position['height'].mean()
            previous_year_avg_weight = players_per_position['weight'].mean()

            teams_df.loc[((teams_df['tmID'] == tmID) & (team_data['year'] == year)), f'previous_year_avg_height_pos_{position}'] = players_per_position['height'].mean()
            teams_df.loc[((teams_df['tmID'] == tmID) & (team_data['year'] == year)), f'previous_year_avg_weight_pos_{position}'] = players_per_position['weight'].mean()

# Merge relevant data

In [159]:
main_df = teams_df

# Merge teams_df with coaches_df
# main_df = pd.merge(teams_df, coaches_df, on=['year', 'tmID'], how='left')

# Merge teams_df with teams_post_df
# main_df = pd.merge(main_df, teams_post_df, on=['year', 'tmID'], how='left')

# Merge with players_teams_df
# main_df = pd.merge(main_df, players_teams_df, on=['year', 'tmID'], how='left')

# Merge with awards_players_df
# main_df = pd.merge(main_df, awards_players_df, on=['playerID','year'], how='left')

# Merge with players_df
# main_df = pd.merge(main_df, players_df, on=['playerID'], how='left')

# Display the resulting merged data frame
main_df.head()

,year,tmID,current_year_rank,playoff,o_fgm,o_fga,o_ftm,o_fta,o_3pm,o_3pa,...,previous_year_avg_height_pos_C,previous_year_avg_weight_pos_C,previous_year_avg_height_pos_C-F,previous_year_avg_weight_pos_C-F,previous_year_avg_height_pos_F,previous_year_avg_weight_pos_F,previous_year_avg_height_pos_G,previous_year_avg_weight_pos_G,previous_year_avg_height_pos_G-F,previous_year_avg_weight_pos_G-F
0,9,ATL,7,N,895,2258,542,725,202,598,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,ATL,2,Y,1089,2428,569,755,114,374,...,79.0,218.0,76.0,186.5,74.400000,175.200000,68.750000,147.250000,73.5,153.0
2,1,CHA,8,N,812,1903,431,577,131,386,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,CHA,4,Y,746,1780,410,528,153,428,...,76.0,215.0,74.5,182.5,71.666667,154.666667,69.000000,158.200000,71.0,145.0
4,3,CHA,2,Y,770,1790,490,663,211,527,...,77.0,219.5,75.0,182.5,70.500000,156.500000,68.833333,142.666667,NaN,NaN


# Drop irrelevant columns
### After merging

In [160]:
# main_df = main_df.drop(["rank", "firstRound", "semis", "finals", "attend", "stint_x", "stint_y", "birthDate", "deathDate", "height", "weight", "award"], axis=1)
# main_df['award'].fillna(False, inplace=True) # mark non-award winners as False
# main_df['team_post_wins'].fillna(0, inplace=True)
# main_df['team_post_losses'].fillna(0, inplace=True)

for column in main_df.filter(regex=("previous_year_avg_.*")).columns.to_list():
    if column.startswith('previous_year_avg_height_pos_') or column.startswith('previous_year_avg_weight_pos_'):
        main_df[column].fillna(-1, inplace=True)

display(main_df)

,year,tmID,current_year_rank,playoff,o_fgm,o_fga,o_ftm,o_fta,o_3pm,o_3pa,...,previous_year_avg_height_pos_C,previous_year_avg_weight_pos_C,previous_year_avg_height_pos_C-F,previous_year_avg_weight_pos_C-F,previous_year_avg_height_pos_F,previous_year_avg_weight_pos_F,previous_year_avg_height_pos_G,previous_year_avg_weight_pos_G,previous_year_avg_height_pos_G-F,previous_year_avg_weight_pos_G-F
0,9,ATL,7,N,895,2258,542,725,202,598,...,-1.0,-1.0,-1.00,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000
1,10,ATL,2,Y,1089,2428,569,755,114,374,...,79.0,218.0,76.00,186.5,74.400000,175.200000,68.750000,147.250000,73.5,153.000000
2,1,CHA,8,N,812,1903,431,577,131,386,...,-1.0,-1.0,-1.00,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000
3,2,CHA,4,Y,746,1780,410,528,153,428,...,76.0,215.0,74.50,182.5,71.666667,154.666667,69.000000,158.200000,71.0,145.000000
4,3,CHA,2,Y,770,1790,490,663,211,527,...,77.0,219.5,75.00,182.5,70.500000,156.500000,68.833333,142.666667,-1.0,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,6,WAS,5,N,847,1968,388,546,181,510,...,75.0,180.0,75.50,192.5,74.000000,168.333333,68.750000,137.500000,71.5,157.500000
138,7,WAS,4,Y,1016,2199,528,715,187,522,...,75.0,180.0,75.50,192.5,72.500000,166.250000,67.200000,133.800000,71.0,160.000000
139,8,WAS,5,N,877,2170,668,839,163,528,...,79.0,170.0,75.50,192.5,71.000000,171.500000,69.250000,154.250000,71.0,173.666667
140,9,WAS,6,N,885,2131,435,659,163,460,...,78.0,210.0,75.50,192.5,72.600000,172.400000,69.250000,154.250000,71.0,157.500000


# Export clean data to a .CSV file

In [161]:
main_df.to_csv("../data/clean/main_df.csv", index=False)
main_df.head(20)

,year,tmID,current_year_rank,playoff,o_fgm,o_fga,o_ftm,o_fta,o_3pm,o_3pa,...,previous_year_avg_height_pos_C,previous_year_avg_weight_pos_C,previous_year_avg_height_pos_C-F,previous_year_avg_weight_pos_C-F,previous_year_avg_height_pos_F,previous_year_avg_weight_pos_F,previous_year_avg_height_pos_G,previous_year_avg_weight_pos_G,previous_year_avg_height_pos_G-F,previous_year_avg_weight_pos_G-F
0,9,ATL,7,N,895,2258,542,725,202,598,...,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
1,10,ATL,2,Y,1089,2428,569,755,114,374,...,79.000000,218.000000,76.000000,186.5,74.400000,175.200000,68.750000,147.250000,73.500000,153.000000
2,1,CHA,8,N,812,1903,431,577,131,386,...,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
3,2,CHA,4,Y,746,1780,410,528,153,428,...,76.000000,215.000000,74.500000,182.5,71.666667,154.666667,69.000000,158.200000,71.000000,145.000000
4,3,CHA,2,Y,770,1790,490,663,211,527,...,77.000000,219.500000,75.000000,182.5,70.500000,156.500000,68.833333,142.666667,-1.000000,-1.000000
5,4,CHA,2,Y,787,1881,456,590,187,517,...,76.000000,204.666667,74.000000,187.5,70.500000,156.500000,68.333333,140.333333,-1.000000,-1.000000
6,5,CHA,5,N,745,1744,436,590,166,459,...,76.000000,199.000000,74.000000,202.5,71.666667,162.666667,68.800000,143.400000,-1.000000,-1.000000
7,6,CHA,6,N,772,1913,447,624,104,301,...,75.333333,183.000000,75.000000,220.0,72.200000,164.600000,69.200000,150.800000,-1.000000,-1.000000
8,7,CHA,6,N,864,2178,552,777,176,544,...,75.000000,202.000000,75.333333,185.0,73.500000,162.250000,68.500000,154.500000,72.000000,160.000000
9,7,CHI,7,N,858,2175,449,643,157,536,...,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000


### Information about the resulting dataset

In [162]:
main_df.info()
main_df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 0 to 141
Data columns (total 55 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   year                              142 non-null    int64  
 1   tmID                              142 non-null    object 
 2   current_year_rank                 142 non-null    int64  
 3   playoff                           142 non-null    object 
 4   o_fgm                             142 non-null    int64  
 5   o_fga                             142 non-null    int64  
 6   o_ftm                             142 non-null    int64  
 7   o_fta                             142 non-null    int64  
 8   o_3pm                             142 non-null    int64  
 9   o_3pa                             142 non-null    int64  
 10  o_oreb                            142 non-null    int64  
 11  o_dreb                            142 non-null    int64  
 12  o_reb   

,year,current_year_rank,o_fgm,o_fga,o_ftm,o_fta,o_3pm,o_3pa,o_oreb,o_dreb,...,previous_year_avg_height_pos_C,previous_year_avg_weight_pos_C,previous_year_avg_height_pos_C-F,previous_year_avg_weight_pos_C-F,previous_year_avg_height_pos_F,previous_year_avg_weight_pos_F,previous_year_avg_height_pos_G,previous_year_avg_weight_pos_G,previous_year_avg_height_pos_G-F,previous_year_avg_weight_pos_G-F
count,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,...,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000
mean,5.302817,4.084507,860.387324,2039.683099,488.338028,651.366197,157.161972,463.014085,330.500000,730.929577,...,56.751761,150.673357,56.500000,140.673709,62.439085,147.019249,58.641228,126.669470,48.311385,110.043192
std,2.917274,2.095226,86.998969,176.879707,70.749372,86.035246,43.736580,116.166119,41.191432,83.378114,...,32.788538,83.567337,33.012584,81.683360,26.535294,62.324182,24.250575,52.128826,33.721789,76.354116
min,1.000000,1.000000,647.000000,1740.000000,333.000000,469.000000,62.000000,205.000000,242.000000,537.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,3.000000,2.000000,794.500000,1908.500000,435.250000,582.750000,128.250000,389.000000,301.250000,653.250000,...,41.625000,165.500000,72.500000,170.625000,72.500000,164.000000,67.200000,141.750000,-1.000000,-1.000000
50%,5.000000,4.000000,864.000000,2025.000000,483.500000,650.000000,157.000000,459.000000,333.500000,724.000000,...,76.000000,187.000000,75.000000,184.250000,73.250000,171.000000,68.333333,146.666667,70.291667,155.000000
75%,8.000000,6.000000,915.000000,2177.500000,539.000000,716.500000,180.750000,528.000000,356.750000,788.000000,...,76.666667,198.916667,75.500000,190.000000,74.000000,176.125000,69.000000,150.787500,72.000000,165.000000
max,10.000000,8.000000,1128.000000,2485.000000,668.000000,882.000000,283.000000,802.000000,452.000000,931.000000,...,79.000000,240.000000,77.000000,220.000000,75.500000,207.500000,70.666667,159.500000,75.000000,185.500000
